In [41]:
import numpy as np
import pandas as pd
import scanpy as sc
import sys
sys.path.append("../utils/")
import preprocessing
import sci_excerpts
import anndata2ri
import rpy2.robjects as ro
import rpy2.rinterface_lib.callbacks

In [4]:
adata = sc.read_h5ad("../../process/pre-intergration/big_data/20241007_merge_all_step0.h5ad")

In [6]:
#del adata.layers["logcounts"]
#adata.write_h5ad("../../process/pre-intergration/big_data/20241007_merge_all_step0.h5ad")

In [13]:
adata.X[1:10,1:10].toarray()

array([[0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [2., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0.]])

In [20]:
adata

AnnData object with n_obs × n_vars = 372369 × 108146
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent_mito', 'integrated_snn_res.0.5', 'seurat_clusters', 'coarse_anno_1', 'RNA_snn_res.0.5', 'id', 'batch', 'ident'
    uns: 'X_name'

In [21]:
adata.obs.drop(['id', 'batch', 'ident'], axis=1, inplace=True)

In [26]:
adata.obs.drop('RNA_snn_res.0.5', axis=1, inplace=True)

In [28]:
adata.obs.drop(['percent_mito', 'integrated_snn_res.0.5', 'seurat_clusters'], axis=1, inplace=True)

In [29]:
for cat in adata.obs.columns:
    if adata.obs[cat].isnull().any():
        print(cat, "has null values")
    elif "nan" in adata.obs[cat]:
        print(cat, "has 'nan' values")

In [35]:
adata.var["gene_symbols"] = adata.var_names

In [36]:
adata = preprocessing.add_cell_annotations(adata, var_index="gene_symbols")

In [37]:
n_genes_pre = adata.shape[1]
sc.pp.filter_genes(adata, min_cells=10)
n_genes_post = adata.shape[1]
print("Number of genes removed: " + str(n_genes_pre - n_genes_post))
print("Number of genes pre-filtering: " + str(n_genes_pre))
print("Number of genes post filtering: " + str(n_genes_post))

Number of genes removed: 52093
Number of genes pre-filtering: 108146
Number of genes post filtering: 56053


In [38]:
adata.write_h5ad("../../process/pre-intergration/big_data/20241007_mergeall_filter_gene_step1.h5ad")